#### Описание принципа работы


##### Задание:
Требуется написать программу, которая по отрывку текста сможет сказать, принадлежит ли он перу Льва Толстого или Ильи Ильфа и Евгения Петрова.

##### Решение:

В данной задаче можно начинать с tf-idf и логистическая регрессия над ними. Дальше пробовать n-граммы и настраивать параметры tf-idf.

Либо другой вариант - это попробовать SVM и bag of words. Тут работает предположение о том, что стилистика двух авторов довольно сильно различается.

<font size="3" color="black" > <input type="checkbox" name="tasks" value= "" /> <font size="2"> Вычистить цифры из текста </font> <br/> </font>
<font size="3" color="black" > <input type="checkbox" name="tasks" value= "" /> <font size="2"> Потюнить параметры через GridSearch </font> <br/> </font>

#### 0. Сбор данных

В качестве данных мы взяли 3 произведения Ильи Ильфа и Евгения Петрова "Золотой телёнок", "Двенадцать стульев", "Одноэтажная Америка" и 1 произведение Льва Николаевича Толстого "Война и мир. Часть 1".

Выкачали тексты и объеденили в один файл тексты Ильи Ильфа и Евгения Петрова, потому что они маленькие и чтобы соотновение объектов обоих классов было примерно одинаковым.

#### 1. Загружаем и вычищаем текст

In [ ]:
#загружаем непрерывный текст

In [1]:
t_str = "Привет, -- 1человек"
ilif_txt = open("ilif.txt", "rb").read()
tols_txt = open("tolstoy.txt", "rb").read()
test_str = "hi man 2 a good"

In [2]:
#очищаем текст от пунктуации
import re
def clear_txt(text):
    text = text.decode('utf-8')
    #убираем пунктуацию
    words = re.findall(ur'(?u)\w+', text)
    #через пробел слова соединяем
    words_w_d = ' '.join(words)
    #убираем цифры
    clear_w = ''.join(i for i in words_w_d if not i.isdigit())
    return clear_w.lower()

In [3]:
cl_ilif_txt = clear_txt(ilif_txt)
cl_tols_txt = clear_txt(tols_txt)
cl_test_str = clear_txt(test_str)

In [9]:
cl_test_str

u'hi man  a good'

In [8]:
#через проблем - длинная строчка
print len(cl_ilif_txt), len(cl_tols_txt)

2406926 2837571


In [10]:
#сплитим весь текст на отдельные слова 
#выкидываем стоп-слова
import nltk

In [ ]:
#nltk.download()

In [11]:
#берем русские стоп-слова
from nltk.corpus import stopwords 
print stopwords.words("russian")

[u'\u0438', u'\u0432', u'\u0432\u043e', u'\u043d\u0435', u'\u0447\u0442\u043e', u'\u043e\u043d', u'\u043d\u0430', u'\u044f', u'\u0441', u'\u0441\u043e', u'\u043a\u0430\u043a', u'\u0430', u'\u0442\u043e', u'\u0432\u0441\u0435', u'\u043e\u043d\u0430', u'\u0442\u0430\u043a', u'\u0435\u0433\u043e', u'\u043d\u043e', u'\u0434\u0430', u'\u0442\u044b', u'\u043a', u'\u0443', u'\u0436\u0435', u'\u0432\u044b', u'\u0437\u0430', u'\u0431\u044b', u'\u043f\u043e', u'\u0442\u043e\u043b\u044c\u043a\u043e', u'\u0435\u0435', u'\u043c\u043d\u0435', u'\u0431\u044b\u043b\u043e', u'\u0432\u043e\u0442', u'\u043e\u0442', u'\u043c\u0435\u043d\u044f', u'\u0435\u0449\u0435', u'\u043d\u0435\u0442', u'\u043e', u'\u0438\u0437', u'\u0435\u043c\u0443', u'\u0442\u0435\u043f\u0435\u0440\u044c', u'\u043a\u043e\u0433\u0434\u0430', u'\u0434\u0430\u0436\u0435', u'\u043d\u0443', u'\u0432\u0434\u0440\u0443\u0433', u'\u043b\u0438', u'\u0435\u0441\u043b\u0438', u'\u0443\u0436\u0435', u'\u0438\u043b\u0438', u'\u043d\u0438', u'\u

In [21]:
#убираем стоп-слова
def clear_stop(text):
    stop_set = set(stopwords.words("russian"))
    #разбиваем на отдельные слова
    splt_txt = text.split()
    #выкидываем стоп-слова
    words = [w for w in splt_txt if not w in stop_set]
    return words

In [22]:
#убираем стоп-слова
def clear_stop_en(text):
    stop_set = set(stopwords.words("english"))
    #разбиваем на отдельные слова
    splt_txt = text.split()
    #выкидываем стоп-слова
    words = [w for w in splt_txt if not w in stop_set]
    return words

In [23]:
cl_test_str_no_stop = clear_stop_en(cl_test_str)

In [24]:
cl_test_str_no_stop

[u'hi', u'man', u'good']

In [25]:
#убираем стоп-слова
#текст - список слов идущих друг за другом
#cl_ilif_txt - текст представлен как чистый список слов идущих друг за другом.
cl_ilif_txt = clear_stop(cl_ilif_txt)
cl_tols_txt = clear_stop(cl_tols_txt)

In [27]:
#сколько важных слов в обоих текстах
print len(cl_ilif_txt), len(cl_tols_txt)

244758 287181


Нужно теперь поделить список слов на батчи.
Пусть по 100 слов - это получается гиперпараметром, который можно подбирать.

In [108]:
words_in_batch = 100

In [109]:
#поделим список слов на батчи
#def chunks(lst_w, num_w):
#    for i in xrange(0, len(lst_w), num_w):
#        yield lst_w[i:i + num_w]
def chunks_b(lst_w, num_w):
    batches = []
    for i in xrange(0, len(lst_w), num_w):
        tmp_b = lst_w[i:i + num_w]
        batches.append(tmp_b)
    return batches

In [110]:
#делим текст из слов на батчи
ilif_batches = chunks_b(cl_ilif_txt, words_in_batch)
tols_batches = chunks_b(cl_tols_txt, words_in_batch)

In [111]:
#посмотрим на сколько батчей разбились тексты
print len(ilif_batches), len(tols_batches)

2448 2872


In [41]:
ilif_batches[0]

In [112]:
#теперь нужно внутри батчей объеденить слова через пробел в строку
#чтобы проще было Bag of words использовать
fin_ilif_b = []
fin_tols_b = []
for btch in ilif_batches:
    fin_ilif_b.append(" ".join(btch))
for btch in tols_batches:
    fin_tols_b.append(" ".join(btch))

In [46]:
#Результирующие батчи текстов для Ильи Ильфа и Евгения Петрова - fin_ilif_b
##Результирующие батчи текстов для Толстого - fin_tols_b

u'\u043d\u0430\u0447\u043d\u0435\u043c \u043d\u0430\u0447\u0430\u043b\u0430 \u0434\u0438\u043b\u043e\u0433\u0438\u0438 \u0438\u0441\u0442\u043e\u0440\u0438\u0438 \u0441\u043e\u0437\u0434\u0430\u043d\u0438\u044f \u043f\u0443\u0431\u043b\u0438\u043a\u0430\u0446\u0438\u0438 \u0434\u0432\u0435\u043d\u0430\u0434\u0446\u0430\u0442\u0438 \u0441\u0442\u0443\u043b\u044c\u0435\u0432 \u0432\u044b\u043c\u044b\u0441\u0435\u043b \u0438\u0441\u0442\u043e\u0440\u0438\u0438 \u043c\u043d\u043e\u0433\u043e\u043a\u0440\u0430\u0442\u043d\u043e \u043f\u0435\u0440\u0435\u0441\u043a\u0430\u0437\u0430\u043d\u043d\u043e\u0439 \u043c\u0435\u043c\u0443\u0430\u0440\u0438\u0441\u0442\u0430\u043c\u0438 \u043b\u0438\u0442\u0435\u0440\u0430\u0442\u0443\u0440\u043e\u0432\u0435\u0434\u0430\u043c\u0438 \u0442\u0440\u0443\u0434\u043d\u043e \u043e\u0442\u0434\u0435\u043b\u0438\u0442\u044c \u0444\u0430\u043a\u0442\u043e\u0432 \u0440\u0435\u0430\u043b\u044c\u043d\u043e\u0441\u0442\u044c \u043c\u0438\u0441\u0442\u0438\u0444\u

In [50]:
type(fin_ilif_b[0])

unicode

#### 2. Применяем Bag of Words и RandomForest

Теперь нам нужно применить Bag Of Words к нашим данным

In [47]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [48]:
#То что мы обработавыли руками, могло сработать из коробки
#Но так более наглядно демонстрируется что мы делаем
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

In [113]:
#Объеденим все в одну матрицу
# Ильи Ильфа и Евгения Петрова - 0
#Толстой - 1
X = np.concatenate([fin_ilif_b, fin_tols_b])
Y = np.concatenate([np.zeros(len(fin_ilif_b)),np.ones(len(fin_tols_b))])

In [114]:
#количество объектов классов получилось сбалансированным
print len(fin_ilif_b), len(fin_tols_b)

2448 2872


In [52]:
#так как у нас нету тестовой выборки
#нужно CV на текущих данных запустить
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import roc_auc_score, accuracy_score

/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [59]:
from sklearn import svm

In [115]:
ss = ShuffleSplit(len(X), n_iter=5, test_size=0.3)

In [116]:
#На обучении делаем Bag of Words
for train_ids, test_ids in ss:
    rf = RandomForestClassifier(n_estimators=100)
    #clf = svm.SVC(probability=True)
    #обучение BoW
    train_data_features = vectorizer.fit_transform(X[train_ids]).toarray()
    print train_data_features[0]
    #обучение алгоритма
    rf.fit(train_data_features, Y[train_ids])
    #clf.fit(train_data_features, Y[train_ids])
    #тест
    test_data_features = vectorizer.transform(X[test_ids]).toarray()
    #предсказания
    preds = rf.predict(test_data_features)
    probs = rf.predict_proba(test_data_features)
    #preds = clf.predict(test_data_features)
    #probs = clf.predict_proba(test_data_features)
    #качество
    print 'ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(Y[test_ids], probs[:,1]), accuracy_score(Y[test_ids], preds))

[0 0 0 ..., 0 0 0]
ROC-AUC: 0.997, ACC: 0.984
[0 0 0 ..., 0 0 0]
ROC-AUC: 0.999, ACC: 0.988
[0 0 0 ..., 0 0 0]
ROC-AUC: 0.999, ACC: 0.989
[0 0 0 ..., 0 0 0]
ROC-AUC: 0.999, ACC: 0.991
[0 0 0 ..., 0 0 0]
ROC-AUC: 0.999, ACC: 0.987


Качество получается слишком подозрительно хорошим. Пробовал обучать и случайный лес и SVM. SVM, который предсказывает вероятности, обучается гораздо дольше, поэтому оставлен вариант с RF.

Возможно такое высокое качество связанно маленьким резмером выборки.

Еще есть подозрения, что в тексте Толстого встречается много французских слов, а в текстах Ильи Ильфа и Евгения Петрова французских слов почти нету, поэтому так хорошо классифицируется. За счет признаков на основе французских слов.

In [118]:
#ради веселья можно посмотреть словарик
#только проблему с кодировкой никто не отменял
vocab = vectorizer.get_feature_names()
print vocab

[u'ah', u'ai', u'aime', u'ami', u'amie', u'andre', u'armee', u'au', u'autres', u'aux', u'avec', u'avez', u'avoir', u'beau', u'bien', u'bon', u'bonne', u'bourienne', u'ca', u'ce', u'cela', u'cet', u'cette', u'cher', u'chere', u'chose', u'comme', u'comment', u'cousin', u'dans', u'de', u'des', u'deux', u'die', u'dieu', u'dire', u'dit', u'dites', u'do', u'du', u'eh', u'elle', u'empereur', u'en', u'encore', u'entre', u'est', u'et', u'etait', u'ete', u'etes', u'etre', u'faire', u'fait', u'faut', u'femme', u'francais', u'general', u'grand', u'guerre', u'homme', u'honneur', u'ii', u'iii', u'il', u'ils', u'iv', u'ix', u'jamais', u'je', u'jeune', u'la', u'le', u'les', u'lle', u'lui', u'ma', u'mais', u'maman', u'me', u'meme', u'mes', u'moi', u'mon', u'monsieur', u'moscou', u'ne', u'nicolas', u'no', u'nom', u'non', u'notre', u'nous', u'oh', u'on', u'ont', u'oui', u'par', u'pas', u'pauvre', u'pere', u'pierre', u'plus', u'pour', u'prince', u'princesse', u'prusse', u'qu', u'quand', u'que', u'qui', u'